In [19]:
import os

import pandas as pd
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [27]:
train = pd.read_csv('all/test.csv')
test = pd.read_csv('all/test.csv')

In [28]:
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [22]:
def clean(df):
    return (df.set_index('PassengerId')
              .assign(complex_ticket=lambda df: df['Ticket'].str.count(' ').astype(bool).astype(int))
              .pipe(lambda df: pd.get_dummies(df, columns=['Sex', 'Pclass', 'Embarked']))
              .fillna(0)
              .assign(num_cabins=lambda df: df['Cabin'].str.count(' ').apply(lambda x: x + 1 if x else 0))
              .drop(['Name', 'Cabin', 'Ticket'], axis=1)
    )

In [23]:
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [24]:
train = clean(train)
test = clean(test)

In [25]:
train.head()

,Age,SibSp,Parch,Fare,complex_ticket,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,num_cabins
PassengerId,,,,,,,,,,,,,,
892,34.5,0,0,7.8292,0,0,1,0,0,1,0,1,0,NaN
893,47.0,1,0,7.0000,0,1,0,0,0,1,0,0,1,NaN
894,62.0,0,0,9.6875,0,0,1,0,1,0,0,1,0,NaN
895,27.0,0,0,8.6625,0,0,1,0,0,1,0,0,1,NaN
896,22.0,1,1,12.2875,0,1,0,0,0,1,0,0,1,NaN


In [26]:
# model = SGDClassifier(loss='log', verbose=3, penalty='none')
model = LogisticRegression(solver='liblinear', penalty='l1')

In [10]:
model.fit(train.drop('Survived', axis=1), train.Survived)
score = model.score(train.drop('Survived', axis=1), train.Survived)
preds = model.predict(test)
print(score)

KeyError: "['Survived'] not found in axis"

In [16]:
test.assign(Survived=preds).Survived.to_frame().to_csv('results.csv')

NameError: name 'test' is not defined